**[PT]** Para executar este ficheiro veja o ficheiro README.md nesta directoria

**[EN]** To run this file: follow instructions in the README.md file in this directory.

# **[PT]** Ordens Religiosas


In [1]:
from timelink.mhk.utilities import get_engine, get_dbnames, get_connection_string
from sqlalchemy import engine, text, MetaData
from sqlalchemy import MetaData,Table, Column, String, ForeignKey
from sqlalchemy.orm import declarative_base, relationship, sessionmaker
mhk = MetaData()
db = 'ucprosop'
connection_string = get_connection_string(db)
engine = get_engine(db)
Session = sessionmaker(bind=engine)
session = Session()
nattr_table = Table('nattributes', mhk, autoload_with=engine)
print('Connected to',db)

import datetime
print(datetime.datetime.now())

Connected to ucprosop
2022-02-03 10:54:50.452887


### Tabela de expressões usadas para determinar a ordem religiosa nas notas ao nome

In [2]:
from ucalumni.extractors import ordens

for expression in sorted(ordens.keys()):
    print(f'{expression:35} {ordens.get(expression)}')

Support functions and data for processing UC alumni loaded!
3ª                                  Ordem Terceira
agostinho                           Ordem de Santo Agostinho
agostinho descalço                  Ordem de Santo Agostinho (Descalços)
agostinhos descalços                Ordem de Santo Agostinho (Descalços)
alcântara                           Ordem de São Pedro de Alcântara
antónio                             Ordem de Santo António
avis                                Ordem de Avis
beneditino                          Ordem de São Bento
bento                               Ordem de São Bento
bernardo                            Ordem de São Bernardo
carmelita                           Ordem do Carmo
carmo                               Ordem do Carmo
colegial do colégio de jesus        Companhia de Jesus
colégio da graça                    Ordem de Santo Agostinho (Graça)
companhia de jesus                  Companhia de Jesus
cristo                              Ordem de Cristo
desc

## Ordens religiosas com menos de 10 ocorrências

In [33]:
from sqlalchemy import select,distinct,func,desc, and_
from ipywidgets import HTML

stmt = select(nattr_table.c.the_value, func.count(distinct(nattr_table.c.id)).label("n")).\
        where(nattr_table.c.the_type == 'ordem-religiosa').\
        group_by(nattr_table.c.the_value).having(func.count()<10).\
        order_by(desc("n"),nattr_table.c.the_value)



print()
#print("Using code tag")
#print()
for row in session.execute(stmt):
        stmt2 = select(nattr_table.c.id,nattr_table.c.name, nattr_table.c.the_date, nattr_table.c.pobs).\
                        where(
                                and_(nattr_table.c.the_type == 'ordem-religiosa', 
                                nattr_table.c.the_value == row.the_value)).\
                        order_by(nattr_table.c.name)
        result = session.execute(stmt2)
        for aluno in result:
                id_link=  f'<a href="https://pesquisa.auc.uc.pt/details?id={aluno.id}">{aluno.id}</a>'
                # display(HTML(f'<code>{row.the_value:24} [{id_link:5}] {aluno.the_date} {aluno.name:44}</code>'))
                print(f'{row.the_value:24} [{aluno.id:5}] {aluno.the_date} {aluno.name:44} https://pesquisa.auc.uc.pt/details?id={aluno.id}')


S. Tomás                 [187529] 1737-10-01 António de Santo Agostinho Brandão           https://pesquisa.auc.uc.pt/details?id=187529
S. Tomás                 [152581] 1720-10-01 Daniel da Fonseca                            https://pesquisa.auc.uc.pt/details?id=152581
S. Tomás                 [229679] 1750-10-01 Diogo do Rosário                             https://pesquisa.auc.uc.pt/details?id=229679
S. Tomás                 [129425] 1724-10-01 Estevão Cardoso Caldeira                     https://pesquisa.auc.uc.pt/details?id=129425
S. Tomás                 [242679] 1712-10-01 Jerónimo Mesquita                            https://pesquisa.auc.uc.pt/details?id=242679
S. Tomás                 [218974] 1763-10-01 José Caetano de São Tomás                    https://pesquisa.auc.uc.pt/details?id=218974
S. Tomás                 [143542] 1721-10-01 José de Almeida                              https://pesquisa.auc.uc.pt/details?id=143542
S. Tomás                 [183531] 1736-10-01 José Mala

## Ordem religiosa em branco

Resulta da deteção de uma palavra que indica pertença a uma ordem, mas não foi possível determinar qual.

Não deve acontecer, nesses casos a ordem religiosa fica igual à expressão em causa (por exemplo "frei" ou "religioso")

In [35]:
from sqlalchemy import select,distinct,func,desc, and_

stmt = select(nattr_table.c.id,nattr_table.c.name,nattr_table.c.the_value, nattr_table.c.the_date, nattr_table.c.aobs).\
        where(
                and_(nattr_table.c.the_type == 'ordem-religiosa', 
                     nattr_table.c.the_value == None)).\
        order_by(nattr_table.c.the_value)

result = session.execute(stmt)
for row in result:
    id_link=  f'<a href="https://pesquisa.auc.uc.pt/details?id={row.id}">{row.id}</a>'
    print(f'{row.the_date:12} {row.name:64} https://pesquisa.auc.uc.pt/details?id={row.id}')

1575-10-05   Bartolomeu de São José                                           https://pesquisa.auc.uc.pt/details?id=218532
1570-10-01   Fernão de Almeida                                                https://pesquisa.auc.uc.pt/details?id=142917
1568-10-01   Estevão da Ascenção                                              https://pesquisa.auc.uc.pt/details?id=148471
1685-10-15   António Dias Bernardes                                           https://pesquisa.auc.uc.pt/details?id=185416
1601-10-22   Sebastião de Brito                                               https://pesquisa.auc.uc.pt/details?id=188840
0000-00-00   José da Trindade Barros                                          https://pesquisa.auc.uc.pt/details?id=273519
1807-06-03   José do Nascimento                                               https://pesquisa.auc.uc.pt/details?id=287172
1567-10-01   Gaspar Quaresma                                                  https://pesquisa.auc.uc.pt/details?id=132223
1622-11-08   Ant